In [17]:
# import libraries

from pandas_datareader import data as pdr
from yahoo_finance import Share as sshare
from yahoo_fin import stock_info as si 
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time

In [18]:
yf.pdr_override()

In [19]:
# Variables
tickers = si.tickers_sp500()

C:\Users\caro_\anaconda3\envs\scanyahoo\lib\site-packages\yahoo_fin\stock_info.py:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sp500["Symbol"] = sp500["Symbol"].str.replace(".", "-")


In [23]:
tickers = [item.replace(".", "-") for item in tickers]
index_name = '^GSPC'

In [25]:
start_date = datetime.datetime.now() - datetime.timedelta(days=365) # this is to setup start time based on a delta rather than fixed time
start_date

datetime.datetime(2020, 3, 22, 19, 53, 56, 398263)

In [26]:
end_date =datetime.date.today()
end_date

datetime.date(2021, 3, 22)

In [29]:
export_list = pd.DataFrame(columns=['stock', 'rs_rating', '50d SMA', '150d SMA', '200d SMA', '52wlow', '52whigh'])
export_list

,stock,rs_rating,50d SMA,150d SMA,200d SMA,52wlow,52whigh


In [32]:
returns_multiples =[]


In [35]:
# Calculate Index Returns

index_df = pdr.get_data_yahoo(index_name, start_date, end_date)
index_df['perc_change'] = index_df['Adj Close'].pct_change()
index_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,perc_change
Date,,,,,,,
2020-03-23,2290.709961,2300.729980,2191.860107,2237.399902,2237.399902,7402180000,NaN
2020-03-24,2344.439941,2449.709961,2344.439941,2447.330078,2447.330078,7547350000,0.093828
2020-03-25,2457.770020,2571.419922,2407.530029,2475.560059,2475.560059,8285670000,0.011535
2020-03-26,2501.290039,2637.010010,2500.719971,2630.070068,2630.070068,7753160000,0.062414
2020-03-27,2555.870117,2615.909912,2520.020020,2541.469971,2541.469971,6194330000,-0.033687


In [36]:
index_return = (index_df['perc_change'] + 1).cumprod()[-1]
index_return

1.7489497937124012

In [38]:
# Find the top 30% performing stocks (relative to the S&P 500)

for ticker in tickers:
    # Download historical data as a csv file for each stock
    df = pdr.get_data_yahoo(ticker, start_date, end_date)
    df.to_csv(f'{ticker}.csv')

    # Calculating returns relative to the market 
    df['perc_change'] = df['Adj Close'].pct_change()
    stock_return = (df['perc_change'] + 1).cumprod()[-1]

    returns_multiple = round((stock_return/ index_return), 2)
    returns_multiples.extend([returns_multiple])

    print (f' Ticker: {ticker}; Returns Multiple against S&P 500: {returns_multiple}\n')
    time.sleep(1)



ltiple against S&P 500: 0.81

[*********************100%***********************]  1 of 1 completed
 Ticker: NI; Returns Multiple against S&P 500: 0.67

[*********************100%***********************]  1 of 1 completed
 Ticker: NKE; Returns Multiple against S&P 500: 1.26

[*********************100%***********************]  1 of 1 completed
 Ticker: NLOK; Returns Multiple against S&P 500: 0.75

[*********************100%***********************]  1 of 1 completed
 Ticker: NLSN; Returns Multiple against S&P 500: 1.27

[*********************100%***********************]  1 of 1 completed
 Ticker: NOC; Returns Multiple against S&P 500: 0.67

[*********************100%***********************]  1 of 1 completed
 Ticker: NOV; Returns Multiple against S&P 500: 0.86

[*********************100%***********************]  1 of 1 completed
 Ticker: NOW; Returns Multiple against S&P 500: 1.05

[*********************100%***********************]  1 of 1 completed
 Ticker: NRG; Returns Multiple against 

IndexError: index -1 is out of bounds for axis 0 with size 0